<a href="https://colab.research.google.com/github/ReidelVichot/LC_identification/blob/main/USASpendingData_3_26_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import time

# -- this line is to make pandas future-proof, Copy-on-Write will become the default in pandas 3.0.
pd.options.mode.copy_on_write = True

# -- Set the data path
dpath = "/content/drive/MyDrive/Disertation/"
fname = "USASpendingData/PrimeTransactionsAndSubawards_2024-03-28_H20M13S03380006/Assistance_PrimeTransactions_2024-03-28_H20M13S55_1.csv"


In [10]:
df = pd.read_csv(dpath + fname)

<ipython-input-10-57f28d019676>:1: DtypeWarning: Columns (3,31,33,35,36,37,38,39,40,41,43,54,77,81,83,84,86,90,94,96,98) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dpath + fname)


In [4]:
df.head()

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,total_outlayed_amount_for_overall_award,indirect_cost_federal_share_amount,...,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,initial_report_date,last_modified_date
0,8620_NY06C70-7013_-NONE-_14.238_-NONE-,ASST_NON_NY06C70-7013_8620,NY06C70-7013,NaN,NaN,SAI NOT AVAILABLE,247896.0,247896.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-04-30
1,8620_NY01C70-3015_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-3015_8620,NY01C70-3015,NaN,FY2011-TCIR-DOC11442607537-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,313896.0,313896.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-09-16
2,8620_NY01C70-0230_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-0230_8620,NY01C70-0230,NaN,NaN,SAI NOT AVAILABLE,192276.0,192276.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-08-08
3,8620_NY01C60-2011_-NONE-_14.238_-NONE-,ASST_NON_NY01C60-2011_8620,NY01C60-2011,NaN,FY2011-TCIR-DOC11442607531-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,84960.0,84960.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2007-12-19
4,8620_NY01C40-0153_-NONE-_14.238_-NONE-,ASST_NON_NY01C40-0153_8620,NY01C40-0153,NaN,NaN,SAI NOT AVAILABLE,-47771.0,-47771.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-02-26


In [ ]:
# columns of interest:
# federal_action_obligation : Amount of Federal Government's obligation,
#       de-obligation, or liability in dollars for an award transaction. It
#       is what the governemnt promises to pay to non-federal recipients.
# total_obligated_amount : Sum of all the amounts entered in the Action
#       Obligation field.
# indirect_cost_federal_share_amount : Amount of any single Federal award that
#       it is allocated to indirect costs according to the approved budget.
#
# non_federal_funding_amount: Amount of the award funded by non-Federal sources
# total_non_federal_funding_amount: The amount of the total award funded by non-
#       Federal sources.
# action_date : The date the action being reported was issued / signed by the
#       Government or a binding agreement was reached.
# action_date_fiscal_year : Note that the Federal fiscal year begins on
#       October 1 and ends on September 30, thus October 1, 2018 is the
#       first day of the 2019 fiscal year.
# period_of_performance_start_date : The Period of Performance is defined in
#       the 2 CFR 200 as the total estimated time interval between the start of
#       an initial Federal award and the planned end date, which may include
#       one or more funded portions, or budget periods.
# period_of_performance_current_end_date : The contract completion date based
#       on the schedule in the contract. For an initial award, this is the
#       scheduled completion date for the base contract and for any options
#       exercised at time of award.
# assistance_transaction_unique_key : System-generated database key used to
#       uniquely identify each financial assistance transaction record and
#       facilitate record lookup, correction, and deletion. A concatenation
#       of AwardingSubTierAgencyCode, FAIN, URI, AssistanceListingNumber, and
#       AwardModificationAmendmentNumber with a single underscore ('_')
#       character inserted in between each. If a field is blank, it is
#       recorded as "".
#

In [11]:
df = df.drop(columns=['highly_compensated_officer_2_amount',
       'highly_compensated_officer_3_name',
       'highly_compensated_officer_3_amount',
       'highly_compensated_officer_4_name',
       'highly_compensated_officer_4_amount',
       'highly_compensated_officer_5_name',
       'highly_compensated_officer_5_amount', 'usaspending_permalink', 'highly_compensated_officer_1_amount',
       'highly_compensated_officer_2_name'])

In [ ]:
df = df.drop(columns=[])

In [12]:
df.columns

Index(['assistance_transaction_unique_key', 'assistance_award_unique_key',
       'award_id_fain', 'modification_number', 'award_id_uri', 'sai_number',
       'federal_action_obligation', 'total_obligated_amount',
       'total_outlayed_amount_for_overall_award',
       'indirect_cost_federal_share_amount',
       ...
       'business_types_description', 'correction_delete_indicator_code',
       'correction_delete_indicator_description', 'action_type_code',
       'action_type_description', 'record_type_code',
       'record_type_description', 'highly_compensated_officer_1_name',
       'initial_report_date', 'last_modified_date'],
      dtype='object', length=102)

In [13]:
df.head()

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,total_outlayed_amount_for_overall_award,indirect_cost_federal_share_amount,...,business_types_description,correction_delete_indicator_code,correction_delete_indicator_description,action_type_code,action_type_description,record_type_code,record_type_description,highly_compensated_officer_1_name,initial_report_date,last_modified_date
0,8620_NY06C70-7013_-NONE-_14.238_-NONE-,ASST_NON_NY06C70-7013_8620,NY06C70-7013,NaN,NaN,SAI NOT AVAILABLE,247896.0,247896.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2008-04-30
1,8620_NY01C70-3015_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-3015_8620,NY01C70-3015,NaN,FY2011-TCIR-DOC11442607537-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,313896.0,313896.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2008-09-16
2,8620_NY01C70-0230_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-0230_8620,NY01C70-0230,NaN,NaN,SAI NOT AVAILABLE,192276.0,192276.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2008-08-08
3,8620_NY01C60-2011_-NONE-_14.238_-NONE-,ASST_NON_NY01C60-2011_8620,NY01C60-2011,NaN,FY2011-TCIR-DOC11442607531-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,84960.0,84960.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2007-12-19
4,8620_NY01C40-0153_-NONE-_14.238_-NONE-,ASST_NON_NY01C40-0153_8620,NY01C40-0153,NaN,NaN,SAI NOT AVAILABLE,-47771.0,-47771.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,C,NaN,2,NaN,NaN,2017-09-17,2008-02-26
